In [1]:
import os
import torch
# Apply WanDB
import wandb
import argparse
import numpy as np
from copy import deepcopy

from utils import get_dataset, get_net, get_strategy
from pprint import pprint
from hyperparams import *

In [2]:
# Reconfig your WANDB API Key here
os.environ["WANDB_API_KEY"] = WANDB_KEY
os.environ["WANDB_BASE_URL"] = WANDB_HOST

In [3]:
# Create output dir
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
# Login wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: toanpv. Use `wandb login --relogin` to force relogin


True

In [5]:
# Config random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.enabled = False

In [6]:
# device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
dataset_name = "MNIST"
strategy_names = ["RandomSampling", "LeastConfidenceSampling", "MarginSampling", "EntropySampling", "RatioSampling"]

In [8]:
def train_active_learning(dataset, net, strategy):
    # start experiment
    dataset.initialize_labels(N_INIT_LABELED)
    
    # Log to WANDB
    wandb.init(project=WANDB_PROJECT, name=f"{dataset_name} - {strategy_name}")
    wandb.define_metric("acc")
    wandb.define_metric("round")
    
    # round 0 accuracy
    print("Round 0 training...")
    strategy.train()
    preds = strategy.predict(dataset.get_test_data())
    print(f"Round 0 testing accuracy: {dataset.cal_test_acc(preds)}")
    
    wandb.log({
        "round": 0,
        "acc": dataset.cal_test_acc(preds)
    })
    
    # Iterative learning
    for rd in range(1, N_ROUND + 1):
        print(f"Round {rd} training...")

        # query
        query_idxs = strategy.query(N_QUERY)

        # update labels
        strategy.update(query_idxs)
        strategy.train()

        # calculate accuracy
        preds = strategy.predict(dataset.get_test_data())
        acc = dataset.cal_test_acc(preds)
        
        print(f"Round {rd} testing accuracy: {acc}")
        
        wandb.log({
            "round": rd,
            "acc": acc
        })
    
    # Call wandb.finish() when end of experiment
    wandb.finish()

In [ ]:
for strategy_name in strategy_names:
    print(f"RUNNING STRATEGY {strategy_name}")
    # Load dataset
    dataset = get_dataset(dataset_name)
    # Load network
    torch.manual_seed(SEED)
    net = get_net(dataset_name, device) 
    # Load strategy
    strategy = get_strategy(strategy_name)(dataset, net)
    train_active_learning(dataset, net, strategy)

RUNNING STRATEGY RandomSampling


Round 0 training...


100%|███████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.52it/s]


Round 0 testing accuracy: 0.7519
Round 1 training...


100%|███████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.94it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.10it/s]


Round 20 testing accuracy: 0.9443
Round 21 training...


100%|███████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.04it/s]


Round 21 testing accuracy: 0.9486
Round 22 training...


 70%|████████████████████████████████████████████▊                   | 7/10 [00:06<00:02,  1.09it/s]